In [2]:
from bokeh.io import output_notebook, curdoc
output_notebook()
curdoc().theme = "dark_minimal"

Loading BokehJS ...

In [3]:

from bokeh.plotting import figure, show
from bokeh.layouts import column
from bokeh.models import Label
import pandas
from datetime import timedelta

df = pandas.read_table("log.tsv", header=None, names=["Time", "Msg/s", "Cmd/s", "Up", "Left", "Down", "Right", "A", "B", "X", "Y", "Demo", "Anar", "Start"])
df["Time"] = df["Time"].apply(lambda ts: pandas.Timestamp(ts, unit="s", tz="Europe/Paris"))

df["~Msg/s"] = df["Msg/s"].rolling(window=30, center=True).mean()
df["~Cmd/s"] = df["Cmd/s"].rolling(window=30, center=True).mean()

df["~~~Cmd/s"] = df["Cmd/s"].rolling(window=180, center=True).mean()

# cmd_per_msg = df["Cmd/s"] / df["Msg/s"]

# df["Up"] = df["Up"] * cmd_per_msg
# df["Left"] = df["Left"] * cmd_per_msg
# df["Down"] = df["Down"] * cmd_per_msg
# df["Right"] = df["Right"] * cmd_per_msg
# df["A"] = df["A"] * cmd_per_msg
# df["B"] = df["B"] * cmd_per_msg
# df["X"] = df["X"] * cmd_per_msg
# df["Y"] = df["Y"] * cmd_per_msg
# df["Demo"] = df["Demo"] * cmd_per_msg
# df["Anar"] = df["Anar"] * cmd_per_msg
# df["Start"] = df["Start"] * cmd_per_msg

def most_used_cmd(row):

    commands = [
        ("↑", row["Up"]),
        ("←", row["Left"]),
        ("↓", row["Down"]),
        ("→", row["Right"]),
        ("A", row["A"]),
        ("B", row["B"]),
        ("X", row["X"]),
        ("Y", row["Y"]),
        ("DEMO", row["Demo"]),
        ("ANAR", row["Anar"]),
        ("START", row["Start"]),
    ]

    commands.sort(key=lambda t: t[1], reverse=True)
    return commands[0][0]


df["^Cmd"] = df.apply(most_used_cmd, axis=1)


plot = figure(
    x_axis_label="Time", 
    x_axis_type="datetime", 
    x_range=(df["Time"][0], df["Time"][0] + timedelta(minutes=30)), 
    y_axis_label="Activity",
    y_range=(0, 8), 
    sizing_mode="stretch_width", 
    height=600)

# plot.line(df["Time"], df["Msg/s"], color="#246ce0", line_width=1)
# plot.line(df["Time"], df["Cmd/s"], color="#c124e0", line_width=1)
plot.line(df["Time"], df["~Msg/s"], color="#246ce0", line_width=1, legend_label="~Msg/s")
plot.line(df["Time"], df["~Cmd/s"], color="#31bd8e", line_width=1, legend_label="~Cmd/s")
plot.line(df["Time"], df["~~~Cmd/s"], color="#bdb831", line_width=1, line_dash="4 4", legend_label="~~~Cmd/s")

# last_cmd = None
# old_cmd_count = 0

# for time, cmd, msg_s in zip(df["Time"], df["^Cmd"], df["~Msg/s"]):
#     if last_cmd != cmd and cmd in ("DEMO", "ANAR"):
#         old_cmd_count += 1
#         if old_cmd_count >= 4:
#             last_cmd = cmd
#             old_cmd_count = 0
#             plot.add_layout(Label(x=time, y=msg_s + 0.5, text=cmd, text_color="#246ce0"))

handle = show(plot, notebook_handle=True)

